In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from sklearn import svm, metrics
from sklearn.neighbors import KNeighborsClassifier, DistanceMetric
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import train_test_split
from preprocess import preprocess, load_dataset

# Classification of cell types with RNA-seq data

## Preparation

In [ ]:
#Load data and preprocess
datadir = 'data/muraro'
data = load_dataset(datadir, 'muraro')
# datadir = 'data/muris'
# data = load_dataset(datadir, 'tabula-muris')
X, y = preprocess(data)
num_labels = len(np.unique(y))
num_features = X.shape[1]

In [ ]:
X = X.astype(float)
X = np.nan_to_num(X)

In [ ]:
# Train/Test splitting
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.15, shuffle=True)

In [ ]:
# RUN THESE CELLS IF USING TABULA MURIS
store_x_train = X_train # 38206
store_y_train = y_train
store_x_test = X_test # 6743
store_y_test = y_test

In [ ]:
# RUN THESE CELLS IF USING TABULA MURIS
X_train = store_x_train[:7000]
y_train = store_y_train[:7000]
X_test = store_x_test[:1200]
y_test = store_y_test[:1200]

## Algorithms

In [ ]:
def evaluate_classifier(clf, X_train, X_test, y_train, y_test, average='macro'):
    clf.fit(X_train, y_train)
    prediction = clf.predict(X_test)
    precision, recall, f1, support = metrics.precision_recall_fscore_support(y_test, prediction,
                                                                         average=average,
                                                                         zero_division='warn')
    print(f"Classification report for classifier {clf}:\n"
      f"{metrics.classification_report(y_test, prediction)}\n")
    return precision, recall, f1

**Linear SVM**

In [ ]:
from sklearn.model_selection import GridSearchCV
denom = len(X_train[0]) * X_train.var()
param_grid = {'C': [0.1, 1, 10, 100, 1000], 
              'gamma': [1,100000/denom, 10000/denom, 1000/denom, 100/denom, 10/denom, 1/denom],
              'kernel': ['linear']}

grid = GridSearchCV(svm.SVC(), param_grid, refit = True, verbose = 2)
  
# fitting the model for grid search
grid.fit(X_train, y_train)

In [ ]:
print(grid.best_params_)

grid_predictions = grid.predict(X_test)
  
# print classification report
print(metrics.classification_report(y_test, grid_predictions))

**Polynomial SVM**

In [ ]:
param_grid = {'C': [0.1, 1, 10, 100, 1000], 
              'gamma': [2, 1,1000000/denom, 100000/denom, 10000/denom, 1000/denom, 100/denom, 10/denom, 1/denom],
              'kernel': ['poly']}

grid = GridSearchCV(svm.SVC(), param_grid, refit = True, verbose = 2)
  
# fitting the model for grid search
grid.fit(X_train, y_train)

In [ ]:
print(grid.best_params_)

grid_predictions = grid.predict(X_test)
  
# print classification report
print(metrics.classification_report(y_test, grid_predictions))

**RBF SVM**

In [ ]:
param_grid = {'C': [0.1, 1, 10, 100, 1000], 
              'gamma': [1,1000000/denom, 100000/denom, 10000/denom, 1000/denom, 100/denom, 10/denom, 1/denom],
              'kernel': ['rbf']}

grid = GridSearchCV(svm.SVC(), param_grid, refit = True, verbose = 2)
  
# fitting the model for grid search
grid.fit(X_train, y_train)

In [ ]:
print(grid.best_params_)

grid_predictions = grid.predict(X_test)
  
# print classification report
print(metrics.classification_report(y_test, grid_predictions))

**Sigmoid SVM**

In [ ]:
param_grid = {'C': [0.1, 1, 10, 100, 1000], 
              'gamma': [1,1000000/denom, 100000/denom, 10000/denom, 1000/denom, 100/denom, 10/denom, 1/denom],
              'kernel': ['sigmoid']}

grid = GridSearchCV(svm.SVC(), param_grid, refit = True, verbose = 2)
  
# fitting the model for grid search
grid.fit(X_train, y_train)

In [ ]:
print(grid.best_params_)

grid_predictions = grid.predict(X_test)
  
# print classification report
print(metrics.classification_report(y_test, grid_predictions))

In [ ]:
# Untuned classifiers
eval_log = {}

# SVM classifier with linear kernel
print('Working... - linear')
clf = svm.SVC(kernel='linear')
precision, recall, f1 = evaluate_classifier(clf, X_train, X_test, y_train, y_test, average='micro')
eval_log['svm-linear'] = (precision, recall, f1)

# SVM classifier with polynomial kernel
print('Working... - polynomial')
clf = svm.SVC(kernel='poly', degree=3)
precision, recall, f1 = evaluate_classifier(clf, X_train, X_test, y_train, y_test, average='micro')
eval_log['svm-poly'] = (precision, recall, f1)

# SVM classifier with RBF kernel
print('Working... - rbf')
clf = svm.SVC(kernel='rbf')
precision, recall, f1 = evaluate_classifier(clf, X_train, X_test, y_train, y_test, average='micro')
eval_log['svm-rbf'] = (precision, recall, f1)

# SVM classifier with sigmoid kernel
print('Working... - sigmoid')
clf = svm.SVC(kernel='sigmoid')
precision, recall, f1 = evaluate_classifier(clf, X_train, X_test, y_train, y_test, average='micro')
eval_log['svm-sigmoid'] = (precision, recall, f1)

In [ ]:
# kNN classifier with Euclidean distance
print('Working... - euclidean knn')
clf = KNeighborsClassifier(n_neighbors=num_labels, metric='euclidean')
precision, recall, f1 = evaluate_classifier(clf, X_train, X_test, y_train, y_test, average='micro')
eval_log['knn-euclidean'] = (precision, recall, f1)

# kNN classifier with Manhattan distance
print('Working... - manhattan knn')
clf = KNeighborsClassifier(n_neighbors=num_labels, metric='manhattan')
precision, recall, f1 = evaluate_classifier(clf, X_train, X_test, y_train, y_test, average='micro')
eval_log['knn-manhattan'] = (precision, recall, f1)

In [ ]:
# Multi-layer perceptron classifier
clf = MLPClassifier()
precision, recall, f1 = evaluate_classifier(clf, X_train, X_test, y_train, y_test, average='micro')
eval_log['mlp'] = (precision, recall, f1)

In [ ]:
eval_log

## Kernel-based kNN

In [ ]:
def rbf_kernel_dist(x, y, gamma):
    return 1 - np.exp(- gamma * ((x - y) ** 2).sum())

def poly_kernel_dist(x, y, gamma, r=0., d=3):
    Kxx = (r + gamma * (x ** 2).sum()) ** d
    Kyy = (r + gamma * (y ** 2).sum()) ** d
    Kxy = (r + gamma * np.dot(x, y)) ** d
    return Kxx + Kyy - 2 * Kxy

def sigmoid_kernel_dist(x, y, gamma, r=0.):
    Kxx = np.tanh(r + gamma * (x ** 2).sum())
    Kyy = np.tanh(r + gamma * (y ** 2).sum())
    Kxy = np.tanh(r + gamma * np.dot(x, y))
    return Kxx + Kyy - 2 * Kxy

**RBF Kernel KNN**

In [ ]:
param_grid = {'n_neighbors': [num_labels, 3, 5, 11, 19], 
              'metric_params': [{'gamma' : 1}, {'gamma' : 1000000/denom}, {'gamma' : 100000/denom}, {'gamma' : 10000/denom}, 
                                {'gamma' : 1000/denom}, {'gamma' : 100/denom},{'gamma' : 10/denom},{'gamma' : 1/denom}],
              'metric': [rbf_kernel_dist]}

grid = GridSearchCV(KNeighborsClassifier(), param_grid, refit = True, verbose = 2)
  
# fitting the model for grid search
grid.fit(X_train, y_train)

In [ ]:
print(grid.best_params_)

grid_predictions = grid.predict(X_test)
  
# print classification report
print(metrics.classification_report(y_test, grid_predictions))

In [ ]:
# Untuned rbf knn
clf = KNeighborsClassifier(n_neighbors=num_labels, metric=rbf_kernel_dist, 
                           metric_params={'gamma' : 1 / num_features})
clf.fit(X_train, y_train)
prediction = clf.predict(X_test)
print(f"Classification report for classifier {clf}:\n"
      f"{metrics.classification_report(y_test, prediction)}\n")

**Poly Kernel knn**

In [ ]:
param_grid = {'n_neighbors': [num_labels, 3, 5, 11, 19], 
              'metric_params': [{'gamma' : 1, }, {'gamma' : 1000000/denom}, {'gamma' : 100000/denom}, {'gamma' : 10000/denom}, 
                                {'gamma' : 1000/denom}, {'gamma' : 100/denom},{'gamma' : 10/denom},{'gamma' : 1/denom}],
              'metric': [poly_kernel_dist]}

grid = GridSearchCV(KNeighborsClassifier(), param_grid, refit = True, verbose = 2)
  
# fitting the model for grid search
grid.fit(X_train, y_train)

In [ ]:
print(grid.best_params_)

grid_predictions = grid.predict(X_test)
  
# print classification report
print(metrics.classification_report(y_test, grid_predictions))

In [ ]:
# Untuned poly knn
clf = KNeighborsClassifier(n_neighbors=num_labels, metric=poly_kernel_dist, 
                           metric_params={'gamma' : 1 / num_features})
clf.fit(X_train, y_train)
prediction = clf.predict(X_test)
print(f"Classification report for classifier {clf}:\n"
      f"{metrics.classification_report(y_test, prediction)}\n")

**Sigmoid knn Classifier**

In [ ]:
param_grid = {'n_neighbors': [num_labels, 3, 5, 11, 19], 
              'metric_params': [{'gamma' : 1, }, {'gamma' : 1000000/denom}, {'gamma' : 100000/denom}, {'gamma' : 10000/denom}, 
                                {'gamma' : 1000/denom}, {'gamma' : 100/denom},{'gamma' : 10/denom},{'gamma' : 1/denom}],
              'metric': [poly_kernel_dist]}

grid = GridSearchCV(KNeighborsClassifier(), param_grid, refit = True, verbose = 2)
  
# fitting the model for grid search
grid.fit(X_train, y_train)

In [ ]:
print(grid.best_params_)

grid_predictions = grid.predict(X_test)
  
# print classification report
print(metrics.classification_report(y_test, grid_predictions))

In [ ]:
# Untuned sigmoid knn
clf = KNeighborsClassifier(n_neighbors=num_labels, metric=sigmoid_kernel_dist, 
                           metric_params={'gamma' : 1 / num_features})
clf.fit(X_train, y_train)
prediction = clf.predict(X_test)
print(f"Classification report for classifier {clf}:\n"
      f"{metrics.classification_report(y_test, prediction)}\n")